<img src="data/images/lecture-notebook-header.png" />

# Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) is an essential step in the data mining process to ensure reliable results, otherwise: *"garbage in, garbage out"*. It involves the initial examination and exploration of a dataset to understand its main characteristics, patterns, distributions, and relationships before applying more advanced data mining techniques. The primary goals of EDA in data mining are as follows:

* **Data Understanding:** EDA helps in gaining a deeper understanding of the dataset by examining its structure, content, and quality. It involves assessing the size of the dataset, the number of features, data types, and the presence of missing or inconsistent values. EDA also involves summarizing and visualizing the data to identify any initial patterns or trends.

* **Data Visualization:** Visualization techniques, such as histograms, scatter plots, box plots, and heatmaps, are employed in EDA to visually represent the data and reveal its distributional properties, relationships between variables, and potential outliers or anomalies. These visualizations help in identifying patterns, clusters, or trends that may be relevant for subsequent analysis.

* **Descriptive Statistics:** EDA includes computing descriptive statistics, such as mean, median, mode, variance, and standard deviation, to summarize the main characteristics and properties of the dataset. These statistics provide insights into the central tendencies, variabilities, and distributions of the data, allowing for a more comprehensive understanding of the dataset.

* **Data Exploration:** EDA involves exploring different subsets of the data and examining the relationships between variables. This exploration can include identifying correlations, dependencies, or associations between attributes, as well as investigating how different features interact and potentially impact the target variable or outcome of interest.

* **Data Preprocessing:** EDA can highlight the need for data preprocessing steps, such as handling missing values, outliers, or data normalization. By identifying these data quality issues early on, EDA helps guide the appropriate preprocessing techniques required to improve the overall data quality and integrity.

* **Hypothesis Generation:** EDA can generate hypotheses or initial insights about the data that can be further tested and validated using more advanced data mining or statistical techniques. These hypotheses serve as starting points for the subsequent modeling and analysis phases.

Summing up, EDA is all about "getting to know your data", a very early crucial phase in the dats mining. In this notebook, we go through some basic steps of an EDA. The dataset and the results are roughly in line with the examples shown in the slides for Lecture 1.

**Note:** The EDA generally depends on the type of dataset (e.g., records, transactions, text, images, etc.), the task (clustering, classification, regression, association rule mining, etc), and even the specific algorithm for a given task -- for example, data normalization is typically not required when using Decision Trees compared to, e.g., K-Nearest Neighbor classifiers.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Import Required Packages

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

---

## Data Preparation

The dataset used throughout this notebook is directly taken from [Kaggle](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset). This dataset contains 70,000 records of patient data. Each record contains a series of personal and health related information (e.g., age, weight, height, blood pressure, smoker/non-smoker, etc.) The last attribute `cardio` is binary (`1` or `0`) indicating whether the patient is suffering from a cardiovascular disease. As such, the common task here is to predict if an unknown person is suffering from a cardiovascular disease based on his or her own personal and health information.  

**Side note:** Here we assume that the data was collected from a trusted source like a hospital. In practice, **data provenance**, sometimes called data lineage, is a very important consideration for data mining. It refers to a documented trail that accounts for the origin of a piece of data and where it has moved from to where it is presently. Simply speaking, you want to ensure that all data was recorded and handled to (ideally) guarantee the data not only is of good quality to begin with, but also that it has not been purposefully or unpurposefully tampered with. As a data miner, you more commonly get handed data and not record the data yourself. Hence, we do not focus on data provenance here.

### Loading the Data

The dataset is stored in semicolon-separated text `data/datasets/cardio/cardio_train.csv`. You can open the file in the browser or text editor of choice to have a look at the raw file content. Dataset of (highly) structured data are often recorded and transferred using such files. To open the file, we use the `pandas` method [`read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). By default, the method assumes that the attributes are separated by commas. Since in our dataset file the attributes are separated by a semicolon, we have to explicitly specify this.

In [ ]:
# Load file into pandas dataframe
df = pd.read_csv('data/datasets/cardio/cardio_train.csv', sep=';')

# Let's have a look at the first 5 rows
df.head()

Just by looking at the first 5 data points data we can make some observations

* `age` are 5-digits values, so the age is arguably not expressed in years; a look in the documentation states that it's the age in number of days

* Cholesterol levels are commonly measured in (mg/dL). However, the integer values for `cholesterol` indicate that this is a categorical attribute. Again, the documentation clarifies: `1 = normal`, `2 = above normal`, `3 = well above normal`

* In this small sample, `gluc`, `smoke`, `alco` show variations in their values. It might be worth checking if indeed the values are all the same, since in this case respective attributes can be removed (not true in this case; see below).

* `cardio` is a categorical attribute with `cardio=1` representing that a person suffers from a cardiovascular disease. So the prediction whether an unknown person is suffering from a cardiovascular disease is a classification task.

* `id` is an artificial attribute (i.e., not a personal attribute) and should generally be removed before any analysis

**Side note:** Ideally, each dataset comes with a comprehensive description about the meaning of the different attributes and how they have been recorded. You generally do not what to guess how to interpret an attribute and/or its values. For our example dataset here, the description on [Kaggle](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) is very sparse, particularly since we deal with personal and health-related attributes. In practice, it is also very common that you not only receive a dataset but then also discuss with the data collectors to gain some better understanding. For example, while most people know that a (very) high cholesterol level is bad and more likely to result in a cardiovascular disease, this relationship is arguably less obvious for blood pressure. Data miners cannot be experts in all domains, but having some good intuition about the meaning of attributes and values is important.

The code cell below, prints some very basic information about the dataset. The dataset is arguably rather "simple" as there are only 13 attributes.

In [ ]:
num_records, num_attributes = df.shape

print("There are {} data records, each with {} attributes.". format(num_records, num_attributes))

### Messing up the Data

This dataset is a little bit too clean for an interesting EDA -- (arguably) no missing values or other types of noise. For the purpose of this tutorial, we therefore can artificially mess up the data a bit ourselves.

#### Change Unit of an Attribute

All height measurements are in cm; see documentation. Let's pretend that 20% of the heights have been recorded in inches.

In [ ]:
# Get sample of size 20%
df_sample = df.sample(int(num_records/5))

# Convert the height of data points in this sample from "cm" to "inch" (incl. rounding)
df_sample.height = np.round(df_sample.height * 0.393701)

# Update the original dataframe
df.update(df_sample)

#### Add Missing Values

We first add missing data in terms of setting 10% of all weight measurements to 0. In practice, this might reflect that a person was not willing to report his or her weight. Note that such a default value is still a "normal" value compared to a more explicit representation of a missing value such as using NaN; see below.

In [ ]:
# Get sample of size 10%
df_sample = df.sample(int(num_records/10))

# Set weight to 0 across sample
df_sample.weight = 0

# Update the original dataframe
df.update(df_sample)

Additionally, we set 1% of all cells (i.e., row/column combinations) to NA (*Not Available*) or NaN (*Not a Number*). In the code cell below, the snippet `np.random.random(df.shape)` creates a matrix with 70k rows and 13 columns -- i.e., the shape of our dataframe of random values between 0 and 1. Then, for each cell in this matrix with a value lower than `0.01` -- reflecting the 1% -- we set the corresponding entry in the dataframe to NaN.

In [ ]:
df = df.mask(np.random.random(df.shape) < 0.01, other=np.NaN)

**Note:** There are certainly more "sophisticated" ways to mess up the data in a more natural way. But in the context of this notebook, this is more than sufficient to get the goals and challenges of an EDA across.

### Save the Messy Data

Since our steps to mess up the data include randomized steps, let's save the resulting messy dataset so we can reproduce our results later on.

In [ ]:
df.to_csv('data/datasets/cardio/cardio_train_messy.csv', sep=';', index=None)

If not stated otherwise, we assume that `cardio_train_messy.csv` is the dataset we would receive (e.g., download from Kaggle).

---

## Performing EDA

If not stated otherwise, we assume that `cardio_train_messy.csv` is the dataset we would receive (e.g., download from Kaggle).

In [ ]:
# Load file into pandas dataframe
df = pd.read_csv('data/datasets/cardio/cardio_train_messy.csv', sep=';')

# Let's have a look at the first 5 rows
df.head()

**Side note:** You may have noticed that the output differs from the output when printing example records of the original file `cardio_train.csv`; see above. Now many of the original integer values are not floats (e.g., `1.0` instead of `1`). This happened when we set random values to `np.NaN`. Since `np.NaN` is internally represented as float value, `pandas` automatically changed the data types for columns to float here. While there are ways to prevent or fix this, it also causes no harm and again, this can be a common situation to face in practice.

### Missing Values

Missing values general refer to (parts of the) data that are indeed missing. For example, the dataset used in this notebook is a text file where each line represents a data point, i.e., a record, and the attributes of each record are separated by a semicolon, e.g.:

`16.0;;2.0;173.0;60.0;120.0;80.0;1.0;1.0;0.0;0.0;1.0;0.0`

In this example, the second attribute, representing the `age`, is missing.


#### NA Values

The following commands finds all rows that have at least 1 NA value in any column.

In [ ]:
row_has_na = df.isna().any(axis=1)

print(row_has_na)

`row_has_na` is a [`pandas.Series`](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) -- which is roughly equivalent to a column -- with 70 entries containing Boolean values, with `True` indicates that the corresponding row on our dataframe `df` has at least 1 NA value.

Since we only set random NA values in about 1% of our records, you might see only `False` values in the sample output above. However, we can now use `row_has_na` to extract these records with at least 1 NA value and generate a new data frame out of itl see the code cell below.

In [ ]:
df_na = df[row_has_na]

print("There are {} data points that have at least 1 missing value (NA)".format(df_na.shape[0]))

Of course, in practice, we often simply want to remove records with NA values. We can achieve this with a very simply command:

In [ ]:
df_no_na = df[~row_has_na]

print("There are {} data points with no missing values".format(df_no_na.shape[0]))

**Side note:** Blindly removing all records with an NA value anywhere is typically not a good idea. In general, we want to preserve as much data as possible. For example, there is no need to remove records with an NA value for an attribute we do not actually use for a subsequent analysis. In short, handling NA values generally requires some more sophisticated consideration, which we cover a bit more in our Data Preprocessing notebook.

#### Default Values

If no documentation is available or the documentation is lacking this information, it's not obvious if a column might feature a default value with some kind of special interpretation. A basic way to approach is, is to look at the distribution of values for each attribute. In the following, we plot the distribution of the values for `weight`.

In [ ]:
plt.figure()
plt.tick_params(labelsize=14)
plt.hist(df['weight'].to_numpy(), bins=75)
plt.xlabel('weight', fontsize=16)
plt.ylabel('count', fontsize=16)
plt.tight_layout()
plt.show()

Given that the value `0` clearly stands out and no person has a weight of 0, it is not unreasonable to assume that it represents some kind of default value. In practice, of course, this kind of interpretation might be much less obvious. However, in general, we can assume that any default value is "substantially different from normal/common/valid value. For example, it would be a very bad practice to use, say, `100` as a default value for `weight`.

### Identifying Odd Distributions

Looking at the distribution of attribute values can also help to spot oddities in terms of mixed units. For example, if we plot the distribution for `height`, we can easily see the 2 parts reflecting the heights in cm and the heights in inches.

In [ ]:
plt.figure()
plt.tick_params(labelsize=14)
plt.hist(df['height'].to_numpy(), bins=50)
plt.xlabel('height', fontsize=16)
plt.ylabel('count', fontsize=16)
plt.tight_layout()
plt.show()

Again, in practice, these differences can be much less obvious and therefore very difficult to spot. For example, imagine that a much smaller percentage of data points features the "wrong" unit for a data point and/or the scale of the units is much similar (e.g., kilometers vs. miles in case of distances). Note that we also assume that there is indeed something of a "natural" distribution. This is arguably given here since attributes such as the weight or height of a population of people is generally expected to be normally distributed.

For categorical attributes, it is often more straightforward to simply check the set of unique values instead of plotting a distribution -- although this would be perfectly fine as well. Note that we first have to remove all NA in each respective column (if we haven't done so yet). In the code cell below, we use [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) to get the set of unique values of a column.

In [ ]:
print("Unique values of attribute 'gluc':\t", np.unique(df['gluc'][~np.isnan(df['gluc'])]))
print("Unique values of attribute 'alco':\t", np.unique(df['alco'][~np.isnan(df['alco'])]))
print("Unique values of attribute 'active':\t", np.unique(df['active'][~np.isnan(df['active'])]))

Attributes with only 2 values -- particularly if they are `0` and `1` -- indication binary attributes. In our example here, `aloc` and `active` indicate whether a person drinking alcohol or not, and whether the person is active or not. Attribute `gluc` is a bit more interesting. Similar to our intuition about cholesterol levels (see above), we know that glucose levels are measured in, e.g., `mg/dl` and values of 70-100 for a healthy person. This strongly suggests that the values 1, 2, and 3 are class labels. And indeed, the documentation on [Kaggle](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) tells us that 1 = normal, 2 = above normal, and 3 = well above normal.

### Outliers

Outliers refer to observations or data points that significantly deviate from the overall pattern or behavior of the dataset. These data points are distinct from the majority of the other observations and can have a substantial impact on statistical analysis, modeling, and interpretation of results. Outliers can arise due to various reasons, such as measurement errors, experimental anomalies, data entry mistakes, or genuinely rare events. For example, the salary of the CEO of a company can be significantly different from the average salary but it is not an outlier in the sense that this value was incorrectly recorded.

Outliers can be identified through graphical visualization or statistical methods. Common graphical tools include box plots, scatter plots, or histograms, where outliers may appear as points or data values that are far away from the bulk of the data. Statistically, outliers can be detected using techniques such as the z-score, which measures how many standard deviations a data point is away from the mean, or the modified z-score, which considers the median and median absolute deviation for a more robust estimation.

#### Check Outliers in Original Data (i.e., the the units haven't been messed up)

In the original dataset, all values for `height` are recorded in cm, and all values for `weight` are recorded in kg.

In [ ]:
df = pd.read_csv('data/datasets/cardio/cardio_train.csv', sep=';')

# Not really needed since original data has no NA values, but it's a good practice to avoid errors and/or skewed results
df = df.dropna(subset=['height', 'weight'])

Boxplots are a good way to visualize outliers. By default, outliers are all values below `(Q1 - 1.5*IQR)` and above `(W3 + 1.5*IQR)` with

* `Q1`: median of the lower half of the dataset
* `Q3`: median of the upper half of the dataset
* `IQR`: interquartile range with `IQR = Q3-Q1`

But note that the definition of what constitutes an outlier often depends on the dataset and application context.

In [ ]:
X_weight = df['weight']
X_height = df['height']

plt.figure()
plt.tick_params(labelsize=14)
plt.boxplot([X_weight, X_height])
plt.gca().set_xticklabels(['weight (kg)', 'height (cm)'])
plt.tight_layout()
plt.show()

The data point that stands out the most is a height of ~250cm, as this would put the person in the top-10 of tallest people who have ever lived. It's arbitrary unlikely that such a person is in our dataset. We also see very low weights here. While these are expected weights for babies or toddlers, these values would be very likely to be outliers if we assume that our dataset contains only information about adult patients. Again, it is always important to know as much as possible about the dataset to be analyzed.

#### Correlation Outliers

Outliers might manifest not only w.r.t. to a single attribute value but also a combination of attributes. For example, a person might weigh 10 kg (e.g., a baby/toddler) but a combination with a height of 200 cm is clearly some kind of noise. Correlations between only 2 attributes can easily be visualized using scatter plots; see the code cell below for `weight` and `height`.

In [ ]:
X_weight = df['weight']
X_height = df['height']

plt.figure()
plt.tick_params(labelsize=14)
plt.scatter(X_weight, X_height)
plt.xlabel('weight (kg)', fontsize=16)
plt.ylabel('height (cm)', fontsize=16)
plt.tight_layout()
plt.show()

Some observations just by looking at the plot:

* Of course, we can still see outliers w.r.t. to a single attribute like the one data point with a height of about 250 cm, particularly now that this person is supposed to also weight less than 100 kg.

* While babies, toddlers or small children weigh below, say, 20 kg, for a person of average height (~170 cm), this is arguably not survivable.

* Similarly, some persons have a very high weight (>150 kg) but are also very small (<100 cm). One can argue that this most likely represents noise data.

### Class Distribution

Since we already identified this prediction task as a classification task, we can check the distribution of class labels. Ideally, all class labels are more or less equally distributed to make the identification and evaluation of patterns generally easier. But note that this is just a rule of thumb.

In [ ]:
# Get all labels (attribute "cardio") as a numpy array
# (not that there might be NA values for "cardion"; we have to remove those first)
labels = df['cardio'].dropna().to_numpy()

# Calculate all unique labels together with their respective counts
(labels, counts) = np.unique(labels, return_counts=True)

# Normalize the counts
counts = counts / np.sum(counts)

print(counts)

While in this case we can already see that the labels are pretty much perfectly balanced, we can also plot it, for example using a pie chart.

In [ ]:
# Define labels
labels = 'cardio = 0', 'cardio = 1'

fig1, ax1 = plt.subplots()
plt.tick_params(labelsize=24)

patches, texts, autotexts = ax1.pie(counts, labels=labels, autopct='%1.1f%%', shadow=False, startangle=45)

for t in texts:
    t.set_fontsize(18)
for t in autotexts:
    t.set_fontsize(18)    
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.show()

Since we only have 2 class labels, and they are basically perfectly balanced, the pie plot above looks rather boring for this dataset here.

---

## Summary

Exploratory Data Analysis (EDA) plays a crucial role in data mining by enabling analysts to gain insights, understand patterns, and identify trends within a dataset. It involves a set of techniques and visualizations that aid in understanding the structure and characteristics of the data before applying complex modeling or mining algorithms. EDA helps uncover relationships, detect anomalies, and inform feature selection, ultimately leading to more effective data mining processes.

During EDA, analysts begin by examining the basic statistical properties of the dataset, such as mean, median, variance, and distribution. Descriptive statistics provide a summary of the central tendency, spread, and shape of the variables, enabling an initial understanding of the data's overall behavior. Graphical visualizations, such as histograms, box plots, scatter plots, and heatmaps, are then used to visualize the data distribution, identify outliers, and explore potential associations or correlations between variables.

EDA also involves investigating the relationship between different variables through techniques like correlation analysis and cross-tabulation. These techniques help assess the strength and direction of relationships, uncover dependencies, and guide feature selection for subsequent modeling tasks. Additionally, EDA may involve dimensionality reduction techniques, such as principal component analysis (PCA) covered in a later notebook, to explore and visualize high-dimensional datasets in a reduced space, facilitating data understanding and interpretation.

Furthermore, EDA in data mining often involves data preprocessing steps, such as handling missing values, data imputation, normalization, and transformation. These steps ensure data quality and reliability, enabling more accurate mining results. We will cover this more in the Data Preprocessing notebook By thoroughly exploring and understanding the data through EDA, analysts can make informed decisions about appropriate data mining techniques, identify potential challenges or limitations, and formulate relevant hypotheses for further analysis.

In summary, exploratory data analysis in data mining is an essential initial step that helps analysts gain insights, identify patterns, and understand the structure of the data. Through descriptive statistics, visualizations, and correlation analysis, EDA provides a foundation for subsequent modeling and mining tasks. By uncovering relationships, detecting outliers, and preprocessing the data, EDA enables more effective and reliable data mining outcomes. The better you know and understand your data, the more likely you can avoid subpar analysis results due to low data quality. How to fix or clean messy data -- if possible to what extent -- is part of data preprocessing (see notebook).